# WEB SCRAPING ASSIGNMENT 3

<b>Instructions:</b>
- All questions are compulsory.
- In each of the questions you have to automate the process. You do not have to click on any button, click any clickable element, enter keywords in search boxes manually. Each process has to be performed via coding.
- Q1 and Q2 are connected questions i.e. after attempting Q1 proceed to Q2. Do not write whole code from beginning for Q2.
- You may use any web scraping library and tools.
- The question can be attempted in various ways; the correctness of question depends on the output.
- If you encounter any Null values during scraping, you may replace it by hyphen.

In [1]:
# importing libraries
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import selenium
import time
from selenium import webdriver
import os
from PIL import Image
import io
import hashlib

## QUESTION - 1

- Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [7]:
driver = webdriver.Chrome(r"/home/elton/Downloads/chromedriver_linux64/chromedriver")
driver.get('https://www.amazon.in/')

In [8]:
search_bar = driver.find_element_by_id('twotabsearchtextbox')
search_bar.send_keys(str(input("Enter Search Key :")))
search_btn = driver.find_element_by_xpath('//div[@class="nav-search-submit nav-sprite"]')
search_btn.click()

Enter Search Key : Guitar


In [81]:
# Alternate in BS4 :
s_k = input('Enter Search Key : ')
r = requests.get('https://www.amazon.in/s?k='+str(s_k))
soup = BeautifulSoup(r.text, 'lxml')

Enter Search Key :  Violin


## QUESTION - 2

- In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. 

<b>Details to be scraped are:</b><br>"Brand Name", "Name of the Product", "Rating", "No. of
Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“

In [132]:
# https://www.amazon.in/s?k=Violin&page=3
no_pages = 3
s_k = input('Enter Search Key : ') # Configured for 'Violin'

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.in/s?k='+str(s_k)+'&page'+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)

    alls = []
    for d in soup.findAll('div', attrs={'class':'sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 sg-col sg-col-4-of-20'}):
        name = d.find('span', attrs={'class':'a-size-base-plus a-color-base a-text-normal'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('span', attrs={'class':'a-size-base'})
        price = d.find('span', attrs={'class':'a-price-whole'})

        all1=[]

        if name is not None:
            all1.append(name.text)
        else:
            all1.append("-")

        if rating is not None:
            all1.append(rating.text[:3])
        else:
            all1.append('-')

        if users_rated is not None:
            all1.append(users_rated.text)
        else:
            all1.append('-')     

        if price is not None:
            all1.append(price.text)
        else:
            all1.append('-')
        alls.append(all1)    
    return alls


Enter Search Key :  Violin


In [79]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Name','Rating','Customers Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

In [80]:
pd.read_csv("amazon_products.csv")

Name Rating Customers Rated  \
0   Arctic Onyx Violin Kit - Violin 4/4 with case,...    3.7             578   
1   Kadence, Vivaldi 4/4 Violin With Bow, Rosin, H...    3.4             113   
2   PAL MUSIC HOUSE 4/4 Violin With Case bow & Ros...    3.3             196   
3                    PAL MUSIC HOUSE violin with case    3.5              88   
4   Kadence Vivaldi Violin VIV101 Solid Spruce Top...    4.0              47   
..                                                ...    ...             ...   
91  PAL MUSIC HOUSE® 5 string Violin Nylon Bow Ros...    5.0               1   
92  Handcrafted Solid Wood Acoustic Handmade Behal...    2.7               3   
93  ARTIBETTER Violin Mute Metal Violin Practice S...      -               -   
94               SAI MUSICAL Violin (Special Edition)    2.9               4   
95  Kadence, Vivaldi 4/4 Violin With Bow, Rosin, H...    3.4               6   

    Price  
0   3,899  
1   3,999  
2   3,199  
3   2,998  
4   7,499  
..    ...  
91  4,299  
92  3,499  
93   743.  
94  3,200  
95      -  

[96 rows x 4 columns]

## QUESTION - 3
- Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’

In [122]:
DRIVER_PATH = '/home/elton/Downloads/chromedriver_linux64/chromedriver'

def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
        
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"
    wd.get(search_url.format(q=query))

    
    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            try:
                img.click()
            except Exception:
                continue
 
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        results_start = len(thumbnail_results)

    return image_urls

In [123]:
def persist_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=100)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [125]:
if __name__ == '__main__':
    wd = webdriver.Chrome(executable_path=DRIVER_PATH)
    queries = ['fruits', 'cars', 'Machine Learning'] 
    for query in queries:
        wd.get('https://google.com')
        search_box = wd.find_element_by_css_selector('input.gLFyf')
        search_box.send_keys(query)
        links = fetch_image_urls(query,50,wd)
        images_path = '/home/elton/Documents/FlipRobo/web_scraping_assignment-3/images/'
        
        for i in links:
            persist_image(images_path,query,i)
    wd.quit()
    # Scraped 50 outputs 

Found: 100 search results. Extracting links from 0:100
Found: 50 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT4fqXfgTypAMphAwIjfmK5Gvzfzj-5KQVKRA&usqp=CAU - as /home/elton/Documents/FlipRobo/web_scraping_assignment-3/images/fruits/72e5f31be5.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQiBUi0WkFVpmPvA_iCy4j6EXLiI9X-9CkZ3Q&usqp=CAU - as /home/elton/Documents/FlipRobo/web_scraping_assignment-3/images/fruits/e15b19a422.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcShxATPdwjA6uXd_4sGX5cUfhumrDXT2LcaZQ&usqp=CAU - as /home/elton/Documents/FlipRobo/web_scraping_assignment-3/images/fruits/f83de77efd.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR6SjYxZXy_JKZ6NhUw1ZlY2uLry91NcHxN6A&usqp=CAU - as /home/elton/Documents/FlipRobo/web_scraping_assignment-3/images/fruits/cb28f472b2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQrze6YgEOUkpBVygJ4

## QUESTION - 4

 - Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1 st page.

<b>Details to be scraped:</b>
<br> “Brand Name”, “Smartphone name”, “Colour”, “RAM”,
“Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display
Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”.<br><br>
Incase if any of the details is missing then replace it by “- “. <br>
Save your results in a dataframe and CSV.

In [138]:
driver = webdriver.Chrome(r"/home/elton/Downloads/chromedriver_linux64/chromedriver")
driver.get('https://www.flipkart.com/')

In [139]:
search_bar = driver.find_element_by_xpath('//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')
search_bar.send_keys("pixel 4A")
search_btn = driver.find_element_by_xpath('//button[@class="L0Z3Pu"]')
search_btn.click()

In [205]:
data = []
titles = driver.find_elements_by_xpath('//div[@class="_4rR01T"]')
ratings = driver.find_elements_by_xpath('//div[@class="_3LWZlK"]')
rrs = driver.find_elements_by_xpath('//ul[@class="_1xgFaf"]//li[1]')
Screens = driver.find_elements_by_xpath('//ul[@class="_1xgFaf"]//li[2]')
cams = driver.find_elements_by_xpath('//ul[@class="_1xgFaf"]//li[3]')
batts = driver.find_elements_by_xpath('//ul[@class="_1xgFaf"]//li[4]')

for i in range(0,24):
    data.append({'Title':titles[i].text,
                 'Rating': ratings[i].text,
                 'RAM / ROM':rrs[i].text,
                 'Display':Screens[i].text,'Camera':cams[i].text,
                 'Battery':batts[i].text})



In [208]:
df = pd.DataFrame(data)
df.to_csv('flip_results_smp_q4.csv', index=False, encoding='utf-8')
df.head()

Title Rating  \
0  Google Pixel 4a (Just Black, 128 GB)    4.5   
1     REDMI Note 9 (Pebble Grey, 64 GB)    4.3   
2      REDMI Note 9 (Aqua Green, 64 GB)    4.3   
3    REDMI Note 9 (Shadow Black, 64 GB)    4.3   
4    REDMI 9 Prime (Matte Black, 64 GB)    4.4   

                                       RAM / ROM  \
0                          6 GB RAM | 128 GB ROM   
1  4 GB RAM | 64 GB ROM | Expandable Upto 512 GB   
2  4 GB RAM | 64 GB ROM | Expandable Upto 512 GB   
3  4 GB RAM | 64 GB ROM | Expandable Upto 512 GB   
4  4 GB RAM | 64 GB ROM | Expandable Upto 512 GB   

                                 Display  \
0  14.76 cm (5.81 inch) Full HD+ Display   
1  16.59 cm (6.53 inch) Full HD+ Display   
2  16.59 cm (6.53 inch) Full HD+ Display   
3  16.59 cm (6.53 inch) Full HD+ Display   
4  16.59 cm (6.53 inch) Full HD+ Display   

                                       Camera           Battery  
0       12.2MP Rear Camera | 8MP Front Camera  3140 mAh Battery  
1  48MP + 8MP + 2MP + 2MP | 13MP Front Camera  5020 mAh Battery  
2  48MP + 8MP + 2MP + 2MP | 13MP Front Camera  5020 mAh Battery  
3  48MP + 8MP + 2MP + 2MP | 13MP Front Camera  5020 mAh Battery  
4   13MP + 8MP + 5MP + 2MP | 8MP Front Camera  5020 mAh Battery

## QUESTION - 5

- Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [41]:
driver = webdriver.Chrome(r"/home/elton/Downloads/chromedriver_linux64/chromedriver")
driver.get('https://www.google.com/maps/');

In [44]:
s_l = str(input("Enter Location :"))
search_loc = driver.find_element_by_id('searchboxinput')
search_loc.send_keys(s_l)
search_b = driver.find_element_by_id('searchbox-searchbutton')
search_b.click()

Enter Location : Texas


In [46]:
url = driver.current_url
spt = url.split("/")
print("Latitude, Longitude of ",s_l," is : ",spt[6][1:-3])
driver.close()

Latitude, Longitude of  Texas  is :  30.8724652,-109.105186


## QUESTION - 6
 - Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 - September 20) from trak.in.

In [3]:
driver = webdriver.Chrome(r"/home/elton/Downloads/chromedriver_linux64/chromedriver")
driver.get('https://www.trak.in/');

In [6]:
driver.find_element_by_id('menu-item-51510').click()

In [52]:
tables = ['tablepress-50','tablepress-49','tablepress-48']
inf = []
try:
    for i in tables:
        t = driver.find_element_by_xpath('//table[@id="'+str(i)+'"]')
        for rows in range(2,12,2):
            tr = t.find_element_by_xpath('.//tr[@class="row-'+str(rows)+' even"]')
            index = tr.find_element_by_xpath('.//td[@class="column-'+str(1)+'"]')
            date = tr.find_element_by_xpath('.//td[@class="column-'+str(2)+'"]')
            s_name = tr.find_element_by_xpath('.//td[@class="column-'+str(3)+'"]')
            i_v = tr.find_element_by_xpath('.//td[@class="column-'+str(4)+'"]')
            s_v = tr.find_element_by_xpath('.//td[@class="column-'+str(5)+'"]')
            Location = tr.find_element_by_xpath('.//td[@class="column-'+str(6)+'"]')
            i_n = tr.find_element_by_xpath('.//td[@class="column-'+str(7)+'"]')
            i_t = tr.find_element_by_xpath('.//td[@class="column-'+str(8)+'"]')
            amt = tr.find_element_by_xpath('.//td[@class="column-'+str(9)+'"]')
            
            inf.append({'Index':index.text,
                        'Date':date.text,
                        'Startup Name':s_name.text,
                        'Industry / Vertical':i_v.text,
                        'Sub Vertical': s_v.text,
                        'Location':Location.text,
                        'Invertors Name':i_n.text, 
                        'Investment Type': i_t.text,
                        'Amount': amt.text })
        for rows in range(3,12,2):
            tr = t.find_element_by_xpath('.//tr[@class="row-'+str(rows)+' odd"]')
            index = tr.find_element_by_xpath('.//td[@class="column-'+str(1)+'"]')
            date = tr.find_element_by_xpath('.//td[@class="column-'+str(2)+'"]')
            s_name = tr.find_element_by_xpath('.//td[@class="column-'+str(3)+'"]')
            i_v = tr.find_element_by_xpath('.//td[@class="column-'+str(4)+'"]')
            s_v = tr.find_element_by_xpath('.//td[@class="column-'+str(5)+'"]')
            Location = tr.find_element_by_xpath('.//td[@class="column-'+str(6)+'"]')
            i_n = tr.find_element_by_xpath('.//td[@class="column-'+str(7)+'"]')
            i_t = tr.find_element_by_xpath('.//td[@class="column-'+str(8)+'"]')
            amt = tr.find_element_by_xpath('.//td[@class="column-'+str(9)+'"]')
            
            inf.append({'Index':index.text,
                        'Date':date.text,
                        'Startup Name':s_name.text,
                        'Industry / Vertical':i_v.text,
                        'Sub Vertical': s_v.text,
                        'Location':Location.text,
                        'Invertors Name':i_n.text, 
                        'Investment Type': i_t.text,
                        'Amount': amt.text })

except:
    pass

In [67]:
df = pd.DataFrame(inf)
df = df.drop(columns='Index',axis=1)
print(df.info())
df[:10]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Date                 30 non-null     object
 1   Startup Name         30 non-null     object
 2   Industry / Vertical  30 non-null     object
 3   Sub Vertical         30 non-null     object
 4   Location             30 non-null     object
 5   Invertors Name       30 non-null     object
 6   Investment Type      30 non-null     object
 7   Amount               30 non-null     object
dtypes: object(8)
memory usage: 2.0+ KB
None


Date Startup Name                     Industry / Vertical  \
0  08/09/2020       Byju’s                                 EduTech   
1  09/09/2020       Qshala                                 EduTech   
2  09/09/2020  Hippo Video  Video Customer Experience(CX) Platform   
3  07/09/2020          1mg                              E-commerce   
4  31/08/2020         Apna                         Human Resources   
5  12/09/2020    mCaffeine                           Personal Care   
6  02/09/2020        Winzo                           Online Gaming   
7  07/09/2020      Melorra                              E-commerce   
8  31/08/2020        mfine                              HealthTech   
9  03/09/2020      Railofy                          Transportation   

                             Sub Vertical  \
0                         Online Tutoring   
1      Online Curiosity Platform for Kids   
2  Video Customer Experience(CX) Platform   
3                         Online Pharmacy   
4                    Recruitment Platform   
5                     Skincare & Haircare   
6                           Online Gaming   
7                    Online Jewelry Store   
8           On-Demand Healthcare Services   
9            WL & RAC protection platform   

                                     Location  \
0                                   Bangalore   
1                                   Bangalore   
2  Newark, Delaware, United States of Amercia   
3                                     Gurgaon   
4                                   Bangalore   
5                                      Mumbai   
6                                   New Delhi   
7                                   Bangalore   
8                                   Bangalore   
9                                      Mumbai   

                                      Invertors Name Investment Type  \
0  Silver Lake, Tiger Global, General Atlantic an...  Private Equity   
1                                 Rainmatter Capital           Angel   
2  Alpha Wave Incubation, Exfinity Venture Partne...        Series A   
3         Gaja Capital, Tata Capital, Partners Group     In Progress   
4         Lightspeed India and Sequoia Capital India        Series A   
5  Amicus Capital Private Equity I LLP, Amicus Ca...        Series B   
6  Kalaari Capital Partners, IndigoEdge Managemen...        Series B   
7                         Shadow Holdings, Lightbox.  Debt Financing   
8                                   Caretech Pte Inc        Series B   
9                                  Chiratae Ventures            Seed   

           Amount  
0     500,000,000  
1         370,000  
2       4,500,000  
3     100,000,000  
4       8,000,000  
5       3,000,000  
6      15,500,000  
7  upto 8,900,000  
8       5,400,000  
9         950,000

## QUESTION - 7

 - Write a program to scrap all the available details of best gaming laptops from digit.in.

In [68]:
driver = webdriver.Chrome(r"/home/elton/Downloads/chromedriver_linux64/chromedriver")
driver.get('https://www.digit.in/top-products/best-gaming-laptops-40.html');

In [107]:
specs = driver.find_elements_by_xpath('//div[@class="Specs-Wrap"]')
p_list = driver.find_elements_by_xpath('//table[@id="summtable"]//tbody//tr//td[1]')

In [108]:
dat = []
for i in range(7):
    a = specs[i].text.split('\n')    
    dat.append({'Product':p_list[i].text,'OS':a[1],'Display':a[3],'Processor':a[5],'Memory':a[7]})

In [109]:
pd.DataFrame(dat)

Product               OS              Display  \
0     Alienware Area 51M R2  WINDOWS 10 HOME  17.3" (1920 X 1080)   
1          Alienware m15 R3  WINDOWS 10 HOME  15.6" (3840 X 2160)   
2    ASUS ROG Strix Scar 15  WINDOWS 10 HOME  15.6" (1920 X 1080)   
3     Asus ROG Zephyrus G14  WINDOWS 10 HOME    14" (1920 X 1080)   
4          Lenovo Legion 5i   WINDOWS 10 PRO  15.6" (1920 X 1080)   
5  ASUS ROG ZEPHYRUS DUO 15       WINDOWS 10  15.6" (3840 X 1100)   
6      Acer Aspire 7 gaming  WINDOWS 10 HOME  15.6" (1920 X 1080)   

                                           Processor                  Memory  
0   10TH GENERATION INTEL® CORE™ I7-10700 | 2.90 GHZ   1 TB SSD/16 GBGB DDR4  
1       10TH GENERATION INTEL® CORE™ I9-10980HK | NA   1 TB SSD/16 GBGB DDR4  
2                      AMD RYZEN™ 9 5900HX | 3.3 GHZ   1 TB SSD/16 GBGB DDR4  
3               AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ   1 TB SSD/16 GBGB DDR4  
4  10TH GENERATION INTEL® CORE™ I5-10300H | 2.50 GHZ   1 TB SSD/16 GBGB DDR4  
5                 INTEL CORE I7 10TH GEN 10875H | NA  512 GB SSD/4 GBGB DDR4  
6                  AMD RYZEN™ 5-5500U HEXA-CORE | NA  512 GB SSD/8 GBGB DDR4

## QUESTION - 8
 - Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [150]:
driver = webdriver.Chrome(r"/home/elton/Downloads/chromedriver_linux64/chromedriver")
driver.get('https://www.forbes.com/real-time-billionaires/');

In [151]:
tr = driver.find_elements_by_xpath('//tr[@class="base ng-scope"]')

In [154]:
dat = []
for i in range(len(tr)):
    rank = tr[i].find_element_by_xpath('.//td[2]').text
    name = tr[i].find_element_by_xpath('.//td[@class="name"]').text
    nw = tr[i].find_element_by_xpath('.//td[@class="Net Worth"]').text
    age = tr[i].find_element_by_xpath('.//td[@class="age"]').text
    source = tr[i].find_element_by_xpath('.//td[@class="source"]').text
    c_t = tr[i].find_element_by_xpath('.//td[@class="Country/Territory"]').text
    
    dat.append({
        'Rank':rank,
        'Name':name,
        'Net Worth':nw,
        'Age':age,
        'Source':source,
        'Country / Territory':c_t
    })
    
    

In [157]:
pd.DataFrame(dat)

Rank                                   Name Net Worth Age  \
0     1                             Jeff Bezos  $185.1 B  57   
1     2                              Elon Musk  $178.1 B  50   
2     3               Bernard Arnault & family  $174.7 B  72   
3     4                             Bill Gates  $131.2 B  65   
4     5                        Mark Zuckerberg  $128.6 B  37   
5     6                             Larry Page  $118.1 B  48   
6     7                          Larry Ellison  $115.7 B  77   
7     8                            Sergey Brin  $114.1 B  47   
8     9                         Warren Buffett  $103.7 B  90   
9    10  Francoise Bettencourt Meyers & family   $90.3 B  68   
10   11                          Steve Ballmer   $89.1 B  65   
11   12                          Mukesh Ambani   $81.9 B  64   
12   13                         Amancio Ortega   $80.1 B  85   
13   14              Carlos Slim Helu & family   $76.5 B  81   
14   15                             Jim Walton   $69.6 B  73   
15   16                           Alice Walton   $68.6 B  71   
16   17                             Rob Walton   $68.3 B  76   
17   18                         Zhong Shanshan   $65.0 B  66   
18   19                   Phil Knight & family   $60.7 B  83   
19   20                      Michael Bloomberg   $59.0 B  79   
20   21                  Gautam Adani & family   $56.4 B  59   
21   22                        MacKenzie Scott   $53.6 B  51   
22   23                 David Thomson & family   $50.9 B  64   
23   24                           Michael Dell   $50.5 B  56   
24   25              François Pinault & family   $49.6 B  84   

                         Source Country / Territory  
0                        Amazon       United States  
1                 Tesla, SpaceX       United States  
2                          LVMH              France  
3                     Microsoft       United States  
4                      Facebook       United States  
5                        Google       United States  
6                      software       United States  
7                        Google       United States  
8            Berkshire Hathaway       United States  
9                       L'Oréal              France  
10                    Microsoft       United States  
11                  diversified               India  
12                         Zara               Spain  
13                      telecom              Mexico  
14                      Walmart       United States  
15                      Walmart       United States  
16                      Walmart       United States  
17   beverages, pharmaceuticals               China  
18                         Nike       United States  
19                 Bloomberg LP       United States  
20  infrastructure, commodities               India  
21                       Amazon       United States  
22                        media              Canada  
23               Dell computers       United States  
24                 luxury goods              France

## Question - 9

 - Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [11]:
driver = webdriver.Chrome(r"/home/elton/Downloads/chromedriver_linux64/chromedriver")
driver.get('https://www.youtube.com/watch?v=V1Pl8CzNzCw')
driver.maximize_window()
time.sleep(5)

In [12]:
comment_section = driver.find_element_by_xpath('//*[@id="comments"]')
driver.execute_script("arguments[0].scrollIntoView();", comment_section)
time.sleep(7)

In [13]:
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while last_height<60000:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

In [14]:
username_elems = driver.find_elements_by_xpath('//*[@id="author-text"]')
comment_elems = driver.find_elements_by_xpath('//*[@id="content-text"]')
likes = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')
times = driver.find_elements_by_xpath('//*[@class="published-time-text above-comment style-scope ytd-comment-renderer"]')


In [15]:
inf = []
for i in range(0,len(username_elems)):
    un_c = username_elems[i].text
    cm_c = " ".join(comment_elems[i].text.split())
    l = likes[i].text
    t = times[i].text
    
    inf.append({'User Name':un_c,'Comment':cm_c,'Likes':l,'Upload Time':t})

In [16]:
pd.DataFrame(inf)

User Name                                            Comment  \
0    Just A Man of Culture  Billie: "Isn't it lovely, all alone...." Peopl...   
1             Odephs guide  Wow, the perfect duo. What could possibly go w...   
2     @Kalana Madhumadhawa  it's crazy how many people come back to this e...   
3        Jennyfher Rayanny  Só essa música pra aliviar minhas tristezas pr...   
4             Priya Mishra  I don't know why but this song fills me with a...   
..                     ...                                                ...   
538         Lubna Shehadeh  "Need a place to hide, but i can't find one ne...   
539          Masum Prodhan                     I FU*KING LOVE U BILLIE EILISH   
540           لولو الصغيره                                                 ❤️   
541        LUNIME_CHAN UwU                                          Beautiful   
542            Hridoy Khan                                                 ❤️   

    Likes           Upload Time  
0     13K         10 months ago  
1     187            2 days ago  
2     721            2 days ago  
3      32             1 day ago  
4      54          19 hours ago  
..    ...                   ...  
538   251  2 years ago (edited)  
539                   1 day ago  
540                   1 day ago  
541     1            2 days ago  
542                21 hours ago  

[543 rows x 4 columns]

## Question - 10

 - Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [27]:
driver = webdriver.Chrome(r"/home/elton/Downloads/chromedriver_linux64/chromedriver")
driver.get('https://www.hostelworld.com/')
driver.maximize_window()

In [31]:
search = driver.find_element_by_xpath('//input[@id="search-input-field"]')
search.send_keys('London, England')
button = driver.find_element_by_xpath('//*[@id="search-button"]')
button.click()

In [32]:
d = driver.find_elements_by_xpath('//div[@class="property-card"]')

In [140]:
inf = []

for i in range(3,len(d)):
    name = d[i].find_element_by_xpath('.//h2')
    
    try:
        rating = d[i].find_element_by_xpath('.//div[@class="score orange big"]').text
    except: rating = '-'
    
    try:
        O_r = d[i].find_element_by_xpath('.//div[@class="summary orange"]//span[1]').text
    except: O_r = '-'
    
    try:
        T_r = d[i].find_element_by_xpath('.//div[@class="summary orange"]//div[2]').text
    except: T_r = '-'
    
    try:
        P_f = d[i].find_element_by_xpath('.//div[@class="prices-col"]//a//div[1]//div').text
    except: P_f = 'Not Available'
    
    try:
        D_f = d[i].find_element_by_xpath('.//div[@class="prices-col"]//div[2]//div[1]').text
    except: D_f = 'Not Available'
    
    inf.append({
        'Name': name.text,
        'Rating': rating,
        'Overall Rating':O_r,
        'Total Review':T_r,
        'Privates From':P_f,
        'Dorms From':D_f
    })
    
        


In [141]:
pd.DataFrame(inf)

Name Rating Overall Rating  \
0                 London Waterloo Hostel    7.6      Very Good   
1        Wombat's The City Hostel London    9.2         Superb   
2                Prime Backpackers Angel     10         Superb   
3               St Christopher's Village    8.9       Fabulous   
4   PubLove @ The Steam Engine, Waterloo    9.0         Superb   
5         PubLove @ The Crown, Battersea    6.7           Good   
6                  The Dictionary Hostel      -            NEW   
7      Safestay London Elephant & Castle    8.2       Fabulous   
8                   Urbany Hostel London     10         Superb   
9    PubLove @ The Green Man, Paddington      -              -   
10                     Mornington Camden     10         Superb   
11            PubLove @ The Rose & Crown    7.4      Very Good   
12                  New Cross Inn Hostel    7.2      Very Good   
13                    Strand Continental    8.3       Fabulous   
14    St Christopher's Oasis-Female Only    9.1         Superb   
15                      Astor Museum Inn    9.5         Superb   
16                      Astor Kensington    8.7       Fabulous   
17                      Mapesbury Hostel      -              -   
18                       Astor Hyde Park    9.1         Superb   
19                        Phoenix Hostel    8.3       Fabulous   
20    247london Hostel and Private Rooms    6.6           Good   
21                  Burlington Residence    9.8         Superb   
22                              SoHostel    8.6       Fabulous   
23                              Clink261    7.6      Very Good   
24                      Generator London    7.6      Very Good   
25   PubLove @ The White Ferry, Victoria    8.9       Fabulous   
26           Smart Russell Square Hostel    7.3      Very Good   
27            Keystone House Kings Cross    9.1         Superb   
28           Smart Hyde Park View Hostel    8.2       Fabulous   
29               Saint James Backpackers    7.5      Very Good   

           Total Review  Privates From     Dorms From  
0    2409 Total Reviews  Not Available         Rs1520  
1   13140 Total Reviews         Rs8064         Rs1818  
2     513 Total Reviews         Rs6039         Rs2397  
3   10827 Total Reviews         Rs3445         Rs1256  
4     189 Total Reviews        Rs18643         Rs1520  
5     208 Total Reviews         Rs8608         Rs1520  
6       3 Total Reviews         Rs6586  Not Available  
7    3952 Total Reviews         Rs5471         Rs1824  
8     106 Total Reviews         Rs4330         Rs1824  
9                     -        Rs13274         Rs1520  
10     41 Total Reviews  Not Available         Rs3176  
11     99 Total Reviews  Not Available         Rs1520  
12   1004 Total Reviews         Rs3061         Rs1184  
13    773 Total Reviews         Rs4559         Rs1733  
14    594 Total Reviews  Not Available         Rs1408  
15   8714 Total Reviews  Not Available         Rs1824  
16   6326 Total Reviews         Rs7295         Rs1520  
17                    -         Rs4559  Not Available  
18  11326 Total Reviews  Not Available         Rs1520  
19   3420 Total Reviews         Rs2158         Rs1368  
20    443 Total Reviews         Rs5167  Not Available  
21     26 Total Reviews         Rs7131         Rs2067  
22   3960 Total Reviews  Not Available         Rs2026  
23   3172 Total Reviews         Rs5125         Rs1331  
24   6737 Total Reviews         Rs8004         Rs1925  
25    134 Total Reviews  Not Available         Rs1520  
26   9503 Total Reviews  Not Available         Rs1160  
27   8111 Total Reviews         Rs4219  Not Available  
28   4473 Total Reviews         Rs5145         Rs1363  
29   1759 Total Reviews         Rs5526         Rs1925